In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import cv2
import os
import time

In [ ]:
import shutil
try:
    shutil.rmtree('output')
except:
    print('Directory doesnt exist')

Directory doesnt exist


# Unzipping data


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cd drive/MyDrive/'Colab Notebooks'

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/DN.zip"
!rm -r __MACOSX

Archive:  /content/drive/MyDrive/Colab Notebooks/DN.zip
   creating: DN/
  inflating: __MACOSX/._DN           
  inflating: DN/.DS_Store            
  inflating: __MACOSX/DN/._.DS_Store  
   creating: DN/Senenca White Deer Mammal/
  inflating: __MACOSX/DN/._Senenca White Deer Mammal  
   creating: DN/Lady Slipper Orchid Flower/
  inflating: __MACOSX/DN/._Lady Slipper Orchid Flower  
   creating: DN/Pangolin Mammal/
  inflating: __MACOSX/DN/._Pangolin Mammal  
   creating: DN/Spoon Billed Sandpiper Bird/
  inflating: __MACOSX/DN/._Spoon Billed Sandpiper Bird  
   creating: DN/Great Indian Bustard Bird/
  inflating: __MACOSX/DN/._Great Indian Bustard Bird  
   creating: DN/Corpse Flower/
  inflating: __MACOSX/DN/._Corpse Flower  
  inflating: DN/Senenca White Deer Mammal/189.jpg  
  inflating: __MACOSX/DN/Senenca White Deer Mammal/._189.jpg  
  inflating: DN/Senenca White Deer Mammal/162.jpg  
  inflating: __MACOSX/DN/Senenca White Deer Mammal/._162.jpg  
  inflating: DN/Senenca White De

In [ ]:
import os
from os import listdir
for i in listdir('DN/'):
  if i == '.DS_Store':
    continue;
  for j in listdir('DN/' + i):
    if j == '.DS_Store':
      os.remove('DN/' + i + '/' + j)

# Train-Test Split

In [ ]:
%pip install split-folders
import splitfolders
splitfolders.ratio('DN/', output="output", seed=1337, ratio=(.8, 0.1,0.1)) 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Copying files: 1547 files [00:00, 6621.20 files/s]


# Creating class indices

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_generator = ImageDataGenerator()
test_generator = ImageDataGenerator()

In [ ]:
x_train = train_generator.flow_from_directory(r"output/train", target_size = (64,64), class_mode = 'categorical', batch_size = 24)
x_val = train_generator.flow_from_directory(r"output/val", target_size = (64,64), class_mode = 'categorical', batch_size = 24)
x_test = test_generator.flow_from_directory(r"output/test", target_size = (64,64), class_mode = 'categorical', batch_size = 24)

Found 1236 images belonging to 6 classes.
Found 152 images belonging to 6 classes.
Found 159 images belonging to 6 classes.


In [ ]:
x_train.class_indices

{'Corpse Flower': 0,
 'Great Indian Bustard Bird': 1,
 'Lady Slipper Orchid Flower': 2,
 'Pangolin Mammal': 3,
 'Senenca White Deer Mammal': 4,
 'Spoon Billed Sandpiper Bird': 5}

# Creating the model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras import regularizers
import tensorflow, keras

In [ ]:
model = Sequential()
model.add(Convolution2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(300, activation='relu', kernel_regularizer=regularizers.L2(0.001)))
model.add(Dense(150, activation='relu', kernel_regularizer=regularizers.L2(0.001)))
model.add(Dense(75, activation='relu', kernel_regularizer=regularizers.L2(0.001)))
model.add(Dense(6, activation='softmax'))

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 30752)             0         
                                                                 
 dense (Dense)               (None, 300)               9225900   
                                                                 
 dense_1 (Dense)             (None, 150)               45150     
                                                                 
 dense_2 (Dense)             (None, 75)                11325     
                                                        

# Fitting the Model

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
early_stopping = EarlyStopping(monitor='val_accuracy',patience=5)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy',patience=5,factor=0.5,min_lr=0.00001)

callback = [reduce_lr,early_stopping]

In [ ]:
model.fit(x_train, steps_per_epoch=len(x_train), epochs=30, validation_data=x_val, validation_steps=len(x_val))

Epoch 1/30
52/52 [==============================] - 14s 114ms/step - loss: 93.6508 - accuracy: 0.3989 - val_loss: 3.5097 - val_accuracy: 0.6184
Epoch 2/30
52/52 [==============================] - 5s 95ms/step - loss: 1.7948 - accuracy: 0.7872 - val_loss: 2.1069 - val_accuracy: 0.6711
Epoch 3/30
52/52 [==============================] - 5s 94ms/step - loss: 1.0658 - accuracy: 0.9377 - val_loss: 1.9780 - val_accuracy: 0.7368
Epoch 4/30
52/52 [==============================] - 5s 96ms/step - loss: 0.9061 - accuracy: 0.9765 - val_loss: 1.7944 - val_accuracy: 0.7697
Epoch 5/30
52/52 [==============================] - 6s 107ms/step - loss: 0.8845 - accuracy: 0.9814 - val_loss: 2.2730 - val_accuracy: 0.7303
Epoch 6/30
52/52 [==============================] - 5s 94ms/step - loss: 0.9360 - accuracy: 0.9676 - val_loss: 2.0517 - val_accuracy: 0.7566
Epoch 7/30
52/52 [==============================] - 5s 95ms/step - loss: 0.8148 - accuracy: 0.9927 - val_loss: 1.7352 - val_accuracy: 0.7697
Epoch 8/3

In [ ]:
model.save('model_wo_aug.h5')

# Testing the model

In [ ]:
a = model.evaluate_generator(x_test, len(x_test))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


In [ ]:
print(f'Error: {a[0]}')
print(f'Accuracy: {a[1]}')

Error: 2.347017765045166
Accuracy: 0.7232704162597656


In [ ]:
from tensorflow.keras.preprocessing import image

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

classes = {'Corpse Flower': 0,
 'Great Indian Bustard Bird': 1,
 'Lady Slipper Orchid Flower': 2,
 'Pangolin Mammal': 3,
 'Senenca White Deer Mammal': 4,
 'Spoon Billed Sandpiper Bird': 5}
k = 0
n = 0
for cat in listdir('DN/'):
    if cat == '.DS_Store':
      continue;

    for direc in listdir('DN/' + cat):
        if direc == '.DS_Store':
          continue;
        img = image.load_img('DN/' + cat + '/' + direc, target_size = (64,64))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis = 0)
        pred = np.argmax(model.predict(img), axis = 1)
        
        print(cat, pred, classes[cat])
        if pred == classes[cat]:
          k += 1
        
        #img = mpimg.imread('DN/' + cat + '/' + direc)
        #imgplot = plt.imshow(img)
        #plt.show()
        n += 1
print("Accuracy: ", k/n)

Populating the interactive namespace from numpy and matplotlib
1/1 [==============================] - 0s 107ms/step
Lady Slipper Orchid Flower [2] 2
1/1 [==============================] - 0s 15ms/step
Lady Slipper Orchid Flower [2] 2
1/1 [==============================] - 0s 15ms/step
Lady Slipper Orchid Flower [2] 2
1/1 [==============================] - 0s 18ms/step
Lady Slipper Orchid Flower [2] 2
1/1 [==============================] - 0s 21ms/step
Lady Slipper Orchid Flower [2] 2
1/1 [==============================] - 0s 17ms/step
Lady Slipper Orchid Flower [2] 2
1/1 [==============================] - 0s 18ms/step
Lady Slipper Orchid Flower [4] 2
1/1 [==============================] - 0s 18ms/step
Lady Slipper Orchid Flower [2] 2
1/1 [==============================] - 0s 15ms/step
Lady Slipper Orchid Flower [2] 2
1/1 [==============================] - 0s 16ms/step
Lady Slipper Orchid Flower [2] 2
1/1 [==============================] - 0s 14ms/step
Lady Slipper Orchid Flower [2] 2
1

In [ ]:
!pip install ibm_watson_machine_learning 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 34.5 MB/s 
     |████████████████████████████████| 51 kB 909 kB/s 
     |████████████████████████████████| 824 kB 55.2 MB/s 
     |████████████████████████████████| 133 kB 62.0 MB/s 
     |████████████████████████████████| 547 kB 66.1 MB/s 
  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.7.0-py2.py3-none-any.whl size=72563 sha256=2f1468a9b4374d44525ab5d634b23cdaaf7e487064b51179a3a3c03360e84755
  Stored in directory: /root/.cache/pip/wheels/47/22/bf/e1154ff0f5de93cc477acd0ca69abfbb8b799c5b28a66b44c2
  Created wheel for ibm-cos-sdk-core: filename=ibm_cos_sdk_core-2.7.0-py2.py3-none-any.whl size=501013 sha256=2fd1b4d2c30ea7d8f85e0df429b002a4f98df78739bc55d4e24bc71e7d19a51e
  Stored in directory: /root/.cache/pip/wheels/6c/a2/e4/c16d02f809a3ea998e17cfd02c13369281f3d232aaf5902c19
  Created wheel for ibm-cos-sdk-s3transfer: filename=ibm_cos

In [ ]:
!tar -zcvf '/content/drive/MyDrive/Colab Notebooks/dn.tgz' '/content/drive/MyDrive/Colab Notebooks/model_wo_aug.h5'

tar: Removing leading `/' from member names
/content/drive/MyDrive/Colab Notebooks/model_wo_aug.h5


In [ ]:
from ibm_watson_machine_learning import APIClient

WMLCredentials = {
            "url":"https://eu-de.ml.cloud.ibm.com",
            "apikey":"otPqZJ572G9Kvgqd5eTuHSeZ-ktrTokqdXvWd8kaFXLv"
}
client = APIClient(WMLCredentials)

client.set.default_space("12f0d23b-8918-45db-a632-01c21bee5035")

software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

Python 3.7 and 3.8 frameworks are deprecated and will be removed in a future release. Use Python 3.9 framework instead.


'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [ ]:
model_details = client.repository.store_model(model="dn.tgz", meta_props={
client.repository.ModelMetaNames.NAME:"CNN",
client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
})

model_id = client.repository.get_model_id(model_details)

model_id

'7a55f72e-8a35-4f60-891e-ae833db7f9d6'